In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import shutil
import sys   
from glob import glob
import re
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [2]:
import sys
import datetime
from threading import Lock

from prettytable import PrettyTable

def log(str):
    print(str, file=sys.stderr)
    sys.stderr.flush()

class ResultWriter:
    def __init__(self, results_filename):
        self.results_filename = results_filename
        self.lock = Lock()

    def write(self, str):
        self.lock.acquire()
        try:
            with open(self.results_filename +'.txt', "a", encoding="utf-8") as f:
                f.write(str + "\n")
        finally:
            self.lock.release()

    def log(self, msg):
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        msg = timestamp + ": " + msg
        log(msg)
        self.lock.acquire()
        try:
            with open(self.results_filename + ".log", "a", encoding="utf-8") as f:
                f.write(msg + "\n")
        finally:
            self.lock.release()


def get_num_model_parameters(model):
    return sum([p.numel() for p in model.parameters() if p.requires_grad])


def print_model_parameters(model,rw):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, f'{param:,}'])
        total_params += param
    rw.log(f'{table}')
    rw.log(f"Total Trainable Params: {total_params}")
    return total_params

rw = ResultWriter('./logs/CNN_5_10_bert_append_sentence')

In [3]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len,ohe):
        self.tokenizer = tokenizer
        self.df = df
        self.ohe = ohe
        self.title = df['text']
        self.targets = self.ohe.transform(np.array(self.df.label.values).reshape(-1,1)).toarray()
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        if(index < self.__len__()-2):
            text1 = str(self.title[index])
            text2 = str(self.title[index+1])
            text3 = str(self.title[index+2])
            target1 = torch.FloatTensor(self.targets[index])
            target2 = torch.FloatTensor(self.targets[index+1])
            target3 = torch.FloatTensor(self.targets[index+2])
        elif(index == self.__len__()-2):
            text1 = str(self.title[index])
            text2 = str(self.title[index+1])
            text3 = ''
            target1 = torch.FloatTensor(self.targets[index])
            target2 = torch.FloatTensor(self.targets[index+1])
            target3 = torch.FloatTensor(self.ohe.transform(np.array(['NONE']).reshape(-1,1)).toarray()[0])
        elif(index == self.__len__()-1):
            text1 = str(self.title[index])
            text2 = ''
            text3 = ''
            target1 = torch.FloatTensor(self.targets[index])
            target2 = torch.FloatTensor(self.ohe.transform(np.array(['NONE']).reshape(-1,1)).toarray()[0])
            target3 = torch.FloatTensor(self.ohe.transform(np.array(['NONE']).reshape(-1,1)).toarray()[0])

        inputs = self.tokenizer.encode_plus(
            text1+'[SEP]'+text2+'[SEP]'+text3,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        """input2 = self.tokenizer.encode_plus(
            text2,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        input3 = self.tokenizer.encode_plus(
            text3,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )"""

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': [target1,target2,target3]#torch.FloatTensor(self.targets[index])
        }
                
    
    """{
                'input1':{
                    'input_ids': input1['input_ids'].flatten(),
                    'attention_mask': input1['attention_mask'].flatten(),
                    'token_type_ids': input1["token_type_ids"].flatten(),
                    'targets': target1
                },
                'input2':{
                    'input_ids': input2['input_ids'].flatten(),
                    'attention_mask': input2['attention_mask'].flatten(),
                    'token_type_ids': input2["token_type_ids"].flatten(),
                    'targets': target2
                },
                'input3':{
                    'input_ids': input3['input_ids'].flatten(),
                    'attention_mask': input3['attention_mask'].flatten(),
                    'token_type_ids': input3["token_type_ids"].flatten(),
                    'targets': target3
                }
            }"""
            

In [4]:
class BERTClass(torch.nn.Module):
    def __init__(self,pre_trained):
        super(BERTClass, self).__init__()
        self.bert_model = AutoModel.from_pretrained(pre_trained)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear1 = torch.nn.Linear(768, 13)
        self.linear2 = torch.nn.Linear(768, 13)
        self.linear3 = torch.nn.Linear(768, 13)
        self.softmax = torch.nn.Softmax(dim=1)
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output1 = self.linear1(output_dropout)
        output2 = self.linear2(output_dropout)
        output3 = self.linear3(output_dropout)
        return output1,output2,output3

In [5]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [6]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
  val_targets = []
  val_outputs = []

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('# Epoch {}: #'.format(epoch),end='\t')
    for batch_idx, data in enumerate(tqdm(training_loader)):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        target1 = data['targets'][0].to(device,dtype=torch.float)
        target2 = data['targets'][1].to(device,dtype=torch.float)
        target3 = data['targets'][2].to(device,dtype=torch.float)
        
        output1,output2,output3 = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss1 = loss_fn(output1, target1)#focal_loss(outputs, targets)
        loss2 = loss_fn(output2, target2)
        loss3 = loss_fn(output3, target3)
        #print(loss.item())
        #optimizer.zero_grad()
        #loss1.backward()
        #loss2.backward()
        #loss3.backward()
        loss = loss1+loss2+loss3
        loss.backward()
        optimizer.step()
    
        train_loss = train_loss + ((1 / (batch_idx + 1)) * ((loss1.item()+loss2.item()+loss3.item())/3 - train_loss))
        
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(tqdm(validation_loader, 0)):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            #targets = data['targets'].to(device, dtype = torch.float)
            target1 = data['targets'][0].to(device,dtype=torch.float)
            target2 = data['targets'][1].to(device,dtype=torch.float)
            target3 = data['targets'][2].to(device,dtype=torch.float)
        
            output1,output2,output3 = model(ids, mask, token_type_ids)

            loss1 = loss_fn(output1, target1)#focal_loss(outputs, targets)
            loss2 = loss_fn(output2, target2)
            loss3 = loss_fn(output3, target3)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * ((loss1.item()+loss2.item()+loss3.item())/3 - valid_loss))
            val_targets.extend([target1.cpu().detach().numpy().tolist(),target2.cpu().detach().numpy().tolist(),target3.cpu().detach().numpy().tolist()])
            val_outputs.extend([F.softmax(output1,dim=1).cpu().detach().numpy().tolist(),F.softmax(output2,dim=1).cpu().detach().numpy().tolist(),F.softmax(output3,dim=1).cpu().detach().numpy().tolist()])
                  
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      rw.log('epoch:{:.6f} Avg Training Loss: {:.6f} \tAvg Validation Loss: {:.6f}'.format(int(epoch), train_loss, valid_loss))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
      # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        rw.log('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('\t Done\n'.format(epoch))

  return model

In [20]:
#it'll return true and predicted labels
def predict(data_loader,model):
    target_list = []
    output_list = []
    model.eval()
    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(data_loader, 0)):
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          target1 = data['targets'][0].to(device,dtype=torch.float)
          target2 = data['targets'][1].to(device,dtype=torch.float)
          target3 = data['targets'][2].to(device,dtype=torch.float)
          output1,output2,output3 = model(ids, mask, token_type_ids)
          target_list.extend([target1.cpu().detach().numpy().tolist(),target2.cpu().detach().numpy().tolist(),target3.cpu().detach().numpy().tolist()])
          output_list.extend([F.softmax(output1,dim=1).cpu().detach().numpy().tolist(),F.softmax(output2,dim=1).cpu().detach().numpy().tolist(),F.softmax(output3,dim=1).cpu().detach().numpy().tolist()])
    return target_list,output_list#np.array(target_list).argmax(1),np.array(output_list).argmax(1)

In [8]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [9]:
pretrained_name = "bert-base-uncased"

# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
EPOCHS = 5
LEARNING_RATE = 1e-05

rw.log(f'pretrained_name: {pretrained_name}')
rw.log(f'MAX_LEN: {MAX_LEN}')
rw.log(f'BATCH_SIZE: {TRAIN_BATCH_SIZE}')
rw.log(f'EPOCHS: {EPOCHS}')
rw.log(f'LEARNING RATE: {LEARNING_RATE}')

2022-04-27 23:37:21.710: pretrained_name: bert-base-uncased
2022-04-27 23:37:21.713: MAX_LEN: 256
2022-04-27 23:37:21.714: BATCH_SIZE: 8
2022-04-27 23:37:21.715: EPOCHS: 5
2022-04-27 23:37:21.716: LEARNING RATE: 1e-05


In [10]:
glob('./data/*.csv')

['./data\\train_data.csv',
 './data\\train_data_append_label.csv',
 './data\\train_data_append_sentence.csv',
 './data\\val_data.csv',
 './data\\val_data_append_sentence.csv']

In [11]:
train_path = './data/train_data.csv' 
test_path = './data/val_data.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
target_list = list(train_df.label.unique())
target_list.sort()
ohe = OneHotEncoder()
ohe.fit(np.array(target_list).reshape(-1,1))

train_df = train_df.drop(columns=['id','start','end'])
test_df = test_df.drop(columns=['id','start','end'])

train_df, val_df = train_test_split(train_df, test_size=0.2)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [12]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_name)

In [13]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN,ohe)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN,ohe)
test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN,ohe)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=0 )
val_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=0)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
rw.log(f'device: {device}')

2022-04-27 23:37:26.469: device: cuda


In [15]:
classifier_name = 'multi_output_CE_loss'
dir_path = "./trained_weights/" +pretrained_name+'/'+ classifier_name 
rw.log(f'pretrained name: {pretrained_name}')
rw.log(f'classifier name: {classifier_name}')
rw.log(f'trained weights path: {dir_path}')

if not os.path.exists(dir_path):  
  os.makedirs(dir_path)
  print(f"The new directory is created: {dir_path}")
    
ckpt_path = dir_path+"/current_checkpoint.pt"
best_model_path = dir_path+"/best_model.pt"

2022-04-27 23:37:26.488: pretrained name: bert-base-uncased
2022-04-27 23:37:26.489: classifier name: multi_output_CE_loss
2022-04-27 23:37:26.490: trained weights path: ./trained_weights/bert-base-uncased/multi_output_CE_loss


In [16]:
model = BERTClass(pretrained_name)
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [18]:
##Load Model
try:
    if(os.path.exists(best_model_path)):
        model, optimizer, epoch_val, valid_loss_min = load_ckp(best_model_path, model, optimizer)
    elif(os.path.exists(ckpt_path)):
        model, optimizer, epoch_val, valid_loss_min = load_ckp(ckpt_model, model, optimizer)
except:
    print('no model exist')

In [17]:
model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

# Epoch 1: #	

  0%|                                                                                          | 0/482 [00:00<?, ?it/s]C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_targets.extend([F.softmax(target1).cpu().detach().numpy().tolist(),F.softmax(target2).cpu().detach().numpy().tolist(),F.softmax(target3).cpu().detach().numpy().tolist()])
C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_outputs.extend([F.softmax(output1).cpu().detach().numpy().tolist(),F.softmax(output2).cpu().detach().numpy().tolist(),F.softmax(output3).cpu().detach().numpy().tolist()])
100%|███████████████████████████████████████████████████████████████████████████████▋| 480/482 [00:27<00:00, 17.22it/s]C:\Users\shrey

	 Done

# Epoch 2: #	

  0%|                                                                                          | 0/482 [00:00<?, ?it/s]C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_targets.extend([F.softmax(target1).cpu().detach().numpy().tolist(),F.softmax(target2).cpu().detach().numpy().tolist(),F.softmax(target3).cpu().detach().numpy().tolist()])
C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_outputs.extend([F.softmax(output1).cpu().detach().numpy().tolist(),F.softmax(output2).cpu().detach().numpy().tolist(),F.softmax(output3).cpu().detach().numpy().tolist()])
100%|███████████████████████████████████████████████████████████████████████████████▋| 480/482 [00:28<00:00, 16.94it/s]C:\Users\shrey

	 Done

# Epoch 3: #	

...

	 Done

# Epoch 4: #	

  0%|                                                                                          | 0/482 [00:00<?, ?it/s]C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_targets.extend([F.softmax(target1).cpu().detach().numpy().tolist(),F.softmax(target2).cpu().detach().numpy().tolist(),F.softmax(target3).cpu().detach().numpy().tolist()])
C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_outputs.extend([F.softmax(output1).cpu().detach().numpy().tolist(),F.softmax(output2).cpu().detach().numpy().tolist(),F.softmax(output3).cpu().detach().numpy().tolist()])
100%|███████████████████████████████████████████████████████████████████████████████▋| 480/482 [00:28<00:00, 16.81it/s]C:\Users\shrey

	 Done

# Epoch 5: #	

  0%|                                                                                          | 0/482 [00:00<?, ?it/s]C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_targets.extend([F.softmax(target1).cpu().detach().numpy().tolist(),F.softmax(target2).cpu().detach().numpy().tolist(),F.softmax(target3).cpu().detach().numpy().tolist()])
C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\942197752.py:60: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  val_outputs.extend([F.softmax(output1).cpu().detach().numpy().tolist(),F.softmax(output2).cpu().detach().numpy().tolist(),F.softmax(output3).cpu().detach().numpy().tolist()])
100%|███████████████████████████████████████████████████████████████████████████████▋| 480/482 [00:28<00:00, 17.14it/s]C:\Users\shrey

	 Done



In [21]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=0)
val_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=0)
train_true,train_pred = predict(train_data_loader,model)
val_true,val_pred = predict(val_data_loader,model)

100%|██████████████████████████████████████████████████████████████████████████████████| 16/16 [00:25<00:00,  1.57s/it]


In [41]:
train_t = np.asarray(train_true)

C:\Users\shrey satapara\AppData\Local\Temp\ipykernel_69508\802007770.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_t = np.asarray(train_true)


In [52]:
train_t = list()
for i in train_true:
    for j in i:
        train_t.append(j)
train_p = list()
for i in train_pred:
    for j in i:
        train_p.append(j)

In [56]:
train_t = np.array(train_t).argmax(1)
train_p = np.array(train_p).argmax(1)

In [57]:
train_t

array([0, 8, 6, ..., 6, 2, 0], dtype=int64)

In [58]:
train_clf = classification_report(train_t,train_p,zero_division=True,output_dict = True,target_names=target_list)

In [59]:
train_clf

{'ANALYSIS': {'precision': 0.7819106501794975,
  'recall': 0.9752487562189055,
  'f1-score': 0.8679433252158513,
  'support': 16080},
 'ARG_PETITIONER': {'precision': 0.5373068432671082,
  'recall': 0.5445190156599553,
  'f1-score': 0.540888888888889,
  'support': 2235},
 'ARG_RESPONDENT': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 1050},
 'FAC': {'precision': 0.9113798790544255,
  'recall': 0.9692469597754911,
  'f1-score': 0.9394231314104381,
  'support': 8552},
 'ISSUE': {'precision': 0.9121621621621622,
  'recall': 0.4326923076923077,
  'f1-score': 0.5869565217391305,
  'support': 624},
 'NONE': {'precision': 0.8407172995780591,
  'recall': 0.9056818181818181,
  'f1-score': 0.8719912472647703,
  'support': 2640},
 'PREAMBLE': {'precision': 0.8582788251891753,
  'recall': 0.8850681126835075,
  'f1-score': 0.8714676390154968,
  'support': 7561},
 'PRE_NOT_RELIED': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 243},
 'PRE_RELIED': {'preci

In [62]:
val_t = list()
val_p = list()

for i in val_true:
    for j in i:
        val_t.append(j)

for i in val_pred:
    for j in i:
        val_p.append(j)

In [63]:
val_t = np.array(val_t).argmax(1)
val_p = np.array(val_p).argmax(1)

In [64]:
classification_report(val_t,val_p,zero_division=True,output_dict = True,target_names=target_list)

{'ANALYSIS': {'precision': 0.6115541723400116,
  'recall': 0.7664553876157971,
  'f1-score': 0.6802986043492372,
  'support': 4102},
 'ARG_PETITIONER': {'precision': 0.41156462585034015,
  'recall': 0.45318352059925093,
  'f1-score': 0.4313725490196078,
  'support': 534},
 'ARG_RESPONDENT': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 243},
 'FAC': {'precision': 0.615491651205937,
  'recall': 0.6605276256844201,
  'f1-score': 0.6372148859543818,
  'support': 2009},
 'ISSUE': {'precision': 0.8,
  'recall': 0.3404255319148936,
  'f1-score': 0.47761194029850745,
  'support': 141},
 'NONE': {'precision': 0.7020725388601037,
  'recall': 0.7720797720797721,
  'f1-score': 0.7354138398914518,
  'support': 702},
 'PREAMBLE': {'precision': 0.7194805194805195,
  'recall': 0.7351380042462845,
  'f1-score': 0.7272249934365975,
  'support': 1884},
 'PRE_NOT_RELIED': {'precision': 1.0,
  'recall': 0.0,
  'f1-score': 0.0,
  'support': 48},
 'PRE_RELIED': {'precision': 0.40458015

In [ ]:
train_clf = classification_report(train_true,train_pred,zero_division=True,output_dict = True,target_names=target_list)
val_clf = classification_report(val_true,val_pred,zero_division=True,target_names=target_list,output_dict=True)
rw.log(f'{"_"*30}Training Results{"_"*30}\n')
rw.log(classification_report(train_true,train_pred,zero_division=True,target_names=target_list))
rw.log(f'{"_"*30}Val Results{"_"*30}\n')
rw.log(classification_report(val_true,val_pred,zero_division=True,target_names=target_list))

In [21]:
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=0)
test_true,test_pred = predict(test_data_loader,model)
test_clf = classification_report(test_true,test_pred,zero_division=True,target_names=target_list,output_dict=True)
rw.log(f'{"_"*30}Test Results{"_"*30}\n')
rw.log(classification_report(test_true,test_pred,zero_division=True,target_names=target_list))

100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:15<00:00,  1.33s/it]
2022-04-25 11:50:54.744: ______________________________Test Results______________________________

2022-04-25 11:50:54.749:                 precision    recall  f1-score   support

      ANALYSIS       0.71      0.73      0.72       984
ARG_PETITIONER       0.33      0.23      0.27        70
ARG_RESPONDENT       0.25      0.42      0.31        38
           FAC       0.65      0.77      0.71       580
         ISSUE       0.63      0.80      0.71        50
          NONE       0.82      0.93      0.87       190
      PREAMBLE       0.94      0.77      0.85       508
PRE_NOT_RELIED       1.00      0.00      0.00        12
    PRE_RELIED       0.54      0.46      0.49       142
         RATIO       0.32      0.20      0.25        70
           RLC       0.46      0.25      0.32       116
           RPC       0.72      0.84      0.78        91
           STA       0.44  